In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Linear Regression').getOrCreate()

In [2]:
spark

In [3]:
filePath = 'H:\\PySpark\\Tutorial02\\test1.csv'

In [4]:
## Read The dataset
training = spark.read.csv(filePath,header=True,inferSchema=True)

In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

[Age,Experience]----> new feature--->independent feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [8]:
output=featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
type(featureassembler)

pyspark.ml.feature.VectorAssembler

In [11]:
type(output)

pyspark.sql.dataframe.DataFrame

In [12]:
finalized_data=output.select("Independent Features","Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [15]:
### Coefficients
regressor.coefficients

DenseVector([1771.9298, -298.2456])

In [16]:
### Intercepts
regressor.intercept

-21912.28070175451

In [17]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [18]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(6070.175438596507, 41733456.44813812)